In [5]:
# Don't worry, this code takes some time to load the model
from gensim.models import KeyedVectors

w2v_txt_filename = 'w2vec_s100.txt'
model = KeyedVectors.load_word2vec_format(w2v_txt_filename, unicode_errors="ignore")

# we don't want to train the model anymore, so we can keep just its wv part and delete the rest to save memory.
word2vec = model.wv
del model

In [6]:
def similar_words_to_string(related_words):
    return ', '.join(pair[0] for pair in related)

# Finding words similar one to another, you can't do this with simple BoW:
related = word2vec.similar_by_word('alemanha')
print('Palavras similares com Alemanha:\n{}'.format(similar_words_to_string(related)))

related = word2vec.most_similar(positive=['gasolina', 'etanol'])
print('\n\nCombustíveis:\n{}'.format(similar_words_to_string(related)))

related = word2vec.most_similar(positive=['monarquia'], negative=['rei'], topn=5)
print('\n\nAlgo similar à analogia "king - man..." com "monarquia - rei"":\n{}'.format(similar_words_to_string(related)))

Palavras similares com Alemanha:
bélgica, frança, itália, finlândia, dinamarca, holanda, áustria, grã-bretanha, hungria, suiça


Combustíveis:
diesel, hidratado, combustível, gasóleo, álcool, biocombustível, anidro, querosene, glp, gnv


Algo similar à analogia "king - man..." com "monarquia - rei"":
democracia, social-democracia, liberal-democrática, institucionalidade, constituição


In [7]:
# finding odd word

def demo_odd(words):
    odd = word2vec.doesnt_match(words.split())
    print('Entre {}, "{}" é anomalia.'.format(words.split(), odd))

demo_odd('etanol diesel água gasolina')
demo_odd('brasil russia india china coreia')
demo_odd('carvão petróleo peixe gás')

Entre ['etanol', 'diesel', 'água', 'gasolina'], "água" é anomalia.
Entre ['brasil', 'russia', 'india', 'china', 'coreia'], "brasil" é anomalia.
Entre ['carvão', 'petróleo', 'peixe', 'gás'], "peixe" é anomalia.


In [25]:
# getting data in the proper format to train a classifier
from functools import reduce

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def get_phrase_wv(phrase):
    words = phrase.split()
    vectors = [word2vec[word] for word in words if word in word2vec]
    phrase_vector = reduce(lambda x,y: x+y, vectors) * 1.0/len(vectors)
    return phrase_vector

def get_train_test_data():
    X = []
    y = []
    with open('dataset.csv', 'r') as file:
        for line in file:
            text, label = line.strip().split(',')
            vector = get_phrase_wv(text)
            X.append(vector)
            y.append(label)
    
    encoder = LabelEncoder().fit(y)
    y = encoder.transform(y)
    
    X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.1)
    return X_train, X_test, y_train, y_test, encoder

X_train, X_test, y_train, y_test, encoder = get_train_test_data()

In [26]:
from sklearn.svm import SVC

clf = SVC(probability=True, class_weight='balanced', kernel='linear')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.80000000000000004

In [ ]:
try:
    while True:
        msg = input('> ')
            
        label = encoder.inverse_transform(clf.predict([get_phrase_wv(msg)]))
        if label == 'clima':
            response = 'a previsão do tempo é de dia ensolarado até sexta.'
        elif label == 'saldo':
            response = 'seu saldo atual é de R$120.00, você tem R$ 150.00 na poupança.'
        elif label == 'noticias':
            response = 'Capes anuncia a reabertura do programa para doutorado-sanduíche =D'
        elif label == 'saudacao':
            response = 'olá, como posso ajudar?'
        elif label == 'despedida':
            response = 'até a próxima!'
            
        print('[ROBÔ] Beep, bop, %s' % response)
        if label == 'despedida':
            break
        
except KeyboardInterrupt:
    pass #supressing abortion exception

> hey robô
[ROBÔ] Beep, bop, olá, como posso ajudar?
> será que vai esfriar hoje?
[ROBÔ] Beep, bop, a previsão do tempo é de dia ensolarado até sexta.
> ok! vou ter dinheiro pra sair?
[ROBÔ] Beep, bop, seu saldo atual é de R$120.00, você tem R$ 150.00 na poupança.


In [1]:
# Frases interessantes para perguntar:
#     - vai nevar
#     - está nublado
#     - quais as manchetes
#     - tenho como pagar as dívidas esse mês
#     - devo levar um casaco

# Note que nenhuma das palavras-chave (manchete, dívidas, nublado, nevar) não estão no conjunto de dados.
# (Note que não há tratamento para caracteres especias, por isso estes não são usados.)